In [1]:
import os
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()
embeddings = AzureOpenAIEmbeddings(
openai_api_type="azure",
openai_api_version=os.environ["OPENAI_API_EMBEDDING_VERSION"],
openai_api_key=os.environ["OPENAI_API_EMBEDDING_KEY"],
azure_endpoint=os.environ["AZURE_OPENAI_EMBEDDING_ENDPOINT"],
deployment=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"],
model=os.environ["AZURE_OPENAI_EMBEDDING_MODEL"],
validate_base_url=True,
)


In [2]:
from langchain_neo4j import Neo4jVector

url = "bolt://localhost:7687"
username = "neo4j"
password = "password"


vector_store = Neo4jVector.from_documents(
    [], embeddings, url=url, username=username, password=password
)

In [3]:
import logging
# Set up the logger
logging.basicConfig(
    level=logging.INFO,  # Set to DEBUG for detailed logs
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        # logging.FileHandler("scraper.log"),  # Log to a file
        logging.StreamHandler()  # Log to console
    ]
)
logger = logging.getLogger(__name__)

In [4]:
!pip install html2text
import html2text
import aiohttp
import asyncio
from bs4 import BeautifulSoup
from langchain_core.documents import Document
import json

async def parse_url_to_doc(url: str) -> str:
    try:
        timeout = aiohttp.ClientTimeout(total=10)  # Set timeout to 3 seconds
        async with aiohttp.ClientSession() as session:
            async with session.get(url, timeout= timeout) as response:
                if response.status != 200:
                    logger.warning(f"Failed to fetch {url}: Status code {response.status}")
                    return None
                html_content = await response.text()
                soup = BeautifulSoup(html_content, 'html.parser')
                target_div = soup.find('div', class_= "theme-doc-markdown markdown") #langchain

                if not target_div:
                    target_div = soup.find('article') #langraph

                if not target_div:
                    return None

                return Document(page_content=html2text.html2text(str(target_div)), metadata={"source": url})
    except Exception as e:
        logger.error(f"Error in parsing {url}: {e}")
        


In [5]:
with open('webscraping/visited_urls_langgraph.json', 'r') as f:
    urls = json.load(f)

    # Generator function to yield 10 items at a time
    def batch_iterator(iterable, batch_size):
        for i in range(0, len(iterable), batch_size):
            yield iterable[i:i + batch_size]

    batch_size = 10

    # Create the iterator
    iterator = batch_iterator(urls, batch_size)

    documents = []

    # Use the iterator
    for batch in iterator:
        tasks = [parse_url_to_doc(url) for url in batch]
        results = await asyncio.gather(*tasks)
        documents.append(results)
    documents = [item for sublist in documents for item in sublist]
    documents =  [item for item in documents if item != None]

2025-04-04 01:28:30,394 - WARNING - Failed to fetch https://langchain-ai.github.io/langgraph/branching: Status code 404
2025-04-04 01:28:30,402 - WARNING - Failed to fetch https://langchain-ai.github.io/langgraph/autogen-integration-functional: Status code 404
2025-04-04 01:28:30,416 - WARNING - Failed to fetch https://langchain-ai.github.io/langgraph/command: Status code 404
2025-04-04 01:28:30,418 - WARNING - Failed to fetch https://langchain-ai.github.io/langgraph/async: Status code 404
2025-04-04 01:28:30,419 - WARNING - Failed to fetch https://langchain-ai.github.io/langgraph/agentic_concepts: Status code 404
2025-04-04 01:28:30,420 - WARNING - Failed to fetch https://langchain-ai.github.io/langgraph/breakpoints: Status code 404
2025-04-04 01:28:30,422 - WARNING - Failed to fetch https://langchain-ai.github.io/langgraph/autogen-integration: Status code 404
2025-04-04 01:28:35,773 - WARNING - Failed to fetch https://langchain-ai.github.io/langgraph/create-react-agent-memory: Status

In [6]:
# Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(encoding_name="cl100k_base",
    chunk_size=1500, chunk_overlap=200
)
doc_splits = text_splitter.split_documents(documents)

len(doc_splits)

9895

In [9]:
doc_len=0
import tiktoken
for doc in doc_splits:
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(doc.page_content))
    doc_len += num_tokens
    
print(doc_len)

10263075


In [10]:
import time
iterator = batch_iterator(doc_splits, 400)

for doc in iterator:
    try:
        vector_store.add_documents(doc)
    except Exception as ex:
        logger.info(f"{ex} caught, adding a 62 second sleep and retrying again")
        time.sleep(62)
        vector_store.add_documents(doc)

2025-04-04 02:21:36,194 - INFO - HTTP Request: POST https://kanis-m8htxgs7-eastus.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-04-04 02:23:03,367 - INFO - HTTP Request: POST https://kanis-m8htxgs7-eastus.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-04-04 02:23:43,528 - INFO - HTTP Request: POST https://kanis-m8htxgs7-eastus.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-04-04 02:24:32,915 - INFO - HTTP Request: POST https://kanis-m8htxgs7-eastus.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-04-04 02:25:13,115 - INFO - HTTP Request: POST https://kanis-m8htxgs7-eastus.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
2025-04-04 02:26:02,574 - INFO - HT

In [15]:
vector_store.similarity_search_with_relevance_scores("build a swarm agent", k=10)

2025-04-04 18:52:29,890 - INFO - HTTP Request: POST https://kanis-m8htxgs7-eastus.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"


[(Document(metadata={'source': 'https://langchain-ai.github.io/langgraph/tutorials/workflows'}, page_content='[ ](https://github.com/langchain-\nai/langgraph/edit/main/docs/docs/tutorials/workflows/index.md "Edit this\npage")\n\n# Workflows and Agents¶\n\nThis guide reviews common patterns for agentic systems. In describing these\nsystems, it can be useful to make a distinction between "workflows" and\n"agents". One way to think about this difference is nicely explained in\n[Anthropic\'s](https://python.langchain.com/docs/integrations/providers/anthropic/)\n`Building Effective Agents` blog post:\n\n> Workflows are systems where LLMs and tools are orchestrated through\n> predefined code paths. Agents, on the other hand, are systems where LLMs\n> dynamically direct their own processes and tool usage, maintaining control\n> over how they accomplish tasks.\n\nHere is a simple way to visualize these differences:\n\n![Agent Workflow](../../concepts/img/agent_workflow.png)\n\nWhen building ag